In [2]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_03_reduce")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 13:17:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Reduce : 여러개의 값을 하나로 축약
```
RDD.reduce( <func> )

```

In [3]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
from operator import add

In [5]:
sample_rdd.reduce(add)

15

# partition 이 나뉘어 져 있을 때

```
lambda x, y : (x*2) + y
```

1: x=1, y=2   >> (1*2) + 2 = 4
2: x=4, y=3   >> (4*2) + 3 = 11
3: x=11, y=4  >> (11*2) + 4 = 26
4:


In [6]:
##파티션 1개일 때 계산결과 26
sample_rdd1 = spark.parallelize([1,2,3,4])
sample_rdd1.reduce(lambda x,y: (x*2) + y)

26

In [10]:
#파티션 1개일 때
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [12]:
##파티션 2개일 때 계산결과 18
sample_rdd2 = spark.parallelize([1,2,3,4],2)
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

파티션1 
1: 1*2 + 2 = 4

파티션2
1: 3*2 + 4 = 10

최종결합:
x=4, y=10 : 4*2 + 10 = 18

In [13]:
sample_rdd2.reduce(lambda x,y: (x*2) + y)

18

In [ ]:
##파티션 3개일 때
p1 : 1
p2 : 2
p3 : 3,4 > 10
x=3, y=4 : 3*2 + 4 = 10

#p1, p2 결합
x=1, y=2 : 1*2+2 = 4

#4와 p3가 결합
x=4, y=10 : 4*2+10 = 18


In [14]:
sample_rdd3 = spark.parallelize([1,2,3,4],3)
sample_rdd3.glom().collect()

[[1], [2], [3, 4]]

In [15]:
sample_rdd3.reduce(lambda x,y: (x*2) + y)

18

### 리듀스 연산은 순서 의존적

각 파티션 내에서 연산은 독립적으로 실행, 최종 결과는 파티션 간의 결과가 결합

### fold 연산의 활용

```
fold( zeroValue,  <func> )

```

zeroValue : 시작값, 항등원 *일때 1, +일 경우 0


In [16]:
rdd4 = spark.parallelize([2,3,4],4)
rdd4.reduce(lambda x,y : x*y)

24

In [17]:
rdd4.fold(1, lambda x,y: x*y)

24

In [20]:
#빈 RDD의 예외처리
rdd5 = spark.parallelize([])
try :
    rdd5.reduce(lambda a,b : a+b)
except ValueError as e:
    print(f"Reduce Error : {e}")

Reduce Error : Can not reduce() empty RDD


In [21]:
rdd5.fold(0, lambda a,b : a+b)

0

##  reduce(), fold() 비슷한 처리를 합니다.
1. reduce() 는 단순한 RDD 축약연산, 비어있지 않은 RDD 의 경우
2. fold() 는 일반적이고 오류가 없는 연산, 초기값을 적용해서 비어있는 RDD, 파티션의 경우에도 적용할 수 있음.

In [22]:
rdd6 = spark.parallelize([1,"a",2])
rdd6.collect()

[1, 'a', 2]

In [31]:
try:
    rdd6.reduce(lambda x,y: str(x)+str(y))
except TypeError as e:
    print(f"Reduce Error: {e}")

AttributeError: 'str' object has no attribute 'collect'

In [29]:
rdd6.fold("", lambda x,y: str(x) + str(y))

'1a2'

# groupby(), aggregate()

In [6]:
rdd7 = spark.parallelize([1,2,1,1,3,5,5,8])
rdd7.collect()

[1, 2, 1, 1, 3, 5, 5, 8]

In [8]:
result = rdd7.groupBy(lambda x:x%2).collect()

In [9]:
sorted([(x,sorted(y)) for (x,y) in result])

[(0, [2, 8]), (1, [1, 1, 1, 3, 5, 5])]

## RDD.aggregate(zeroValue, func1_partion, func2_combine)

In [10]:
func1 = lambda x,y : (x[0] + y,  x[1] + 1)

In [11]:
func2 = lambda x,y : (x[0] + y[0], x[1] + y[1])

In [13]:
rdd8 = spark.parallelize( [1,2,3,4] , 2)

In [14]:
rdd8.aggregate( (0,0), func1, func2)

(10, 4)

In [15]:
rdd8.glom().collect()

[[1, 2], [3, 4]]

* 원리  

```
p1
x=[1,2], zerovalue=[0,0]
x[0]=0, x[1]=0
x[0]+y = 0+1=1, x[1]+1 = 0+1=1  >> (1,1)
x[0]+y = 1+2=3, x[1]+1 = 1+1=2 >> (3,2)

p2
x=[3,4], zerovalue[0,0]
x[0]=0, x[1]=0
x[0]+y= 0+3=3, x[1]+1= 0+1=1 >> (3,1)
x[0]+y= 3+4=7, x[1]+1= 1+1=2 >> (7,2)
p1,p2 
x=(3,2), y=(7,2)
x[0]+y[0]=3+7=10, x[1]+y[1]=2+2=4 >> (10,4)
```

## key-value RDD (paired RDD)

groupByKey(), reduceByKey()

그룹핑한 후에 특정 트랜스포메이션을 수행, 키가 필요함.  
처리 속도 : reduceByKey > groupByKey

In [16]:
rdd9 = spark.parallelize(
    [
        ("짜장면", 15),
        ("짬뽕", 10),
        ("짜장면", 5),
        ("짬뽕", 20)
    ]
)

In [17]:
rdd9_g = rdd9.groupByKey()
rdd9_g

PythonRDD[22] at RDD at PythonRDD.scala:53

In [19]:
rdd9_g.mapValues(len).collect() # max, min 사용 가능

[('짜장면', 2), ('짬뽕', 2)]

In [20]:
rdd9_g.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [21]:
rdd9_g.mapValues(lambda x: sum(x) / len(x)).collect()

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [22]:
rdd9.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 30)]

# countByKey()
각 키별로 요소들의 갯수를 카운트 > Action

In [23]:
rdd9.countByKey()

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

# KEY()
키만 가진 RDD > transformation

In [24]:
rdd9.keys()

PythonRDD[33] at RDD at PythonRDD.scala:53

In [25]:
rdd9.keys().collect()

['짜장면', '짬뽕', '짜장면', '짬뽕']

# join() : key를 이용한 결합 - outer join, inner join

inner-join : 두 개의 집합에서 함께 존재하는 요소(key)의 집합(default)  
outer-join : 한 쪽에 있고, 다른 쪽에 없을 때 - left, right

In [27]:
join_rdd1 = spark.parallelize(
    [("a",10),
    ("b",20),
    ("c",30)]
)

join_rdd2 = spark.parallelize(
    [("a",10),
    ("b",20),
    ("b",10),]
)

join_rdd1.join(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 10))]

In [28]:
join_rdd1.leftOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('c', (30, None)), ('a', (10, 10))]

In [29]:
join_rdd1.rightOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 10))]

In [31]:
spark.stop()

## ➕ INNER JOIN과 OUTER JOIN

### INNER JOIN
**INNER JOIN**은 두 테이블에서 공통된 키를 기준으로 매칭되는 행만 반환합니다. 두 테이블의 교집합 부분만 가져오는 방식입니다.

#### 예시
- **Table A**:
  | ID | Name |
  |----|------|
  | 1  | Alice|
  | 2  | Bob  |
  | 3  | Carol|

- **Table B**:
  | ID | Age |
  |----|-----|
  | 2  | 30  |
  | 3  | 25  |
  | 4  | 22  |

- **INNER JOIN 결과**:
  | ID | Name  | Age |
  |----|-------|-----|
  | 2  | Bob   | 30  |
  | 3  | Carol | 25  |

두 테이블에서 ID가 공통인 행만 포함됩니다.

### OUTER JOIN
**OUTER JOIN**은 두 테이블에서 매칭되지 않는 행도 포함하여 반환합니다. OUTER JOIN에는 LEFT JOIN, RIGHT JOIN, FULL OUTER JOIN이 있습니다.

#### LEFT JOIN (LEFT OUTER JOIN)
LEFT JOIN은 왼쪽 테이블의 모든 행을 포함하며, 오른쪽 테이블에 매칭되는 행이 있으면 값을 채워 넣습니다. 매칭되는 행이 없으면 NULL을 넣습니다.

- **LEFT JOIN 결과**:
  | ID | Name  | Age  |
  |----|-------|------|
  | 1  | Alice | NULL |
  | 2  | Bob   | 30   |
  | 3  | Carol | 25   |

#### RIGHT JOIN (RIGHT OUTER JOIN)
RIGHT JOIN은 오른쪽 테이블의 모든 행을 포함하며, 왼쪽 테이블에 매칭되는 행이 있으면 값을 채워 넣습니다. 매칭되는 행이 없으면 NULL을 넣습니다.

- **RIGHT JOIN 결과**:
  | ID | Name  | Age  |
  |----|-------|-----|
  | 2  | Bob   | 30  |
  | 3  | Carol | 25  |
  | 4  | NULL  | 22  |

#### FULL OUTER JOIN
FULL OUTER JOIN은 두 테이블의 모든 행을 포함하며, 어느 한쪽에 매칭되는 행이 없으면 NULL을 넣습니다.

- **FULL OUTER JOIN 결과**:
  | ID | Name  | Age  |
  |----|-------|------|
  | 1  | Alice | NULL |
  | 2  | Bob   | 30   |
  | 3  | Carol | 25   |
  | 4  | NULL  | 22   |

이와 같이, OUTER JOIN은 매칭되지 않는 행도 포함하여 두 테이블을 결합할 수 있는 방법을 제공합니다.
각 JOIN 유형은 데이터 분석에서 다양한 요구사항을 충족하기 위해 사용됩니다.

#### CF. 집합 연산과 JOIN 연산
* 집합 연산:  
    UNION, INTERSECTION, SUBTRACTION은 두 집합 간의 요소를 합치거나, 공통 요소를 찾거나, 차집합을 구하는 연산입니다.  
    일반적인 RDD와 키-값 쌍 RDD 모두에서 사용할 수 있으며, 데이터를 필터링하는 목적으로 주로 사용합니다.

* JOIN 연산:  
    두 RDD 간의 공통된 키를 기준으로 데이터를 결합하는 연산입니다.  
    주로 키-값 쌍 RDD에 사용되며, 공통된 키를 기준으로 데이터를 결합하기 위한 목적으로 활용됩니다.